In [1]:
%load_ext autoreload
%autoreload 2

# Get Company

In [2]:
company = Company.objects.get(name="PRE-UHE-ITA")

# Search Tags

## Procurar a SearchTag de level 1 com o nome de Monitoramento e alterar o campo kind para 'select_option' e o campo redirect para ‘MonitoringPlan’. 

In [11]:
from apps.templates.const import search_tag_kinds

In [12]:
search_tag = SearchTag.objects.filter(company=company, level=1, name__icontains="monitoramento").first()

In [13]:
if search_tag:
    search_tag.kind = search_tag_kinds.SELECT_OPTION
    search_tag.redirect = "MonitoringPlan"
    search_tag.save()

## Criar mais três SearchTags de level 2 indicando os tipos do registro e que o componente pai seja o Monitoramento de level 1. 

In [19]:
names = ['Paramêtros Físicos-Químicos', 'Paramêtros Biológicos', 'Paramêtros Específicos']

In [20]:
for name in names:
    tag = SearchTag.objects.create(
        company=company,
        name=name,
        kind=search_tag_kinds.SELECT_OPTION,
        level=2,
        description='',
        redirect='',
    )
    tag.parent_tags.add(search_tag)

# Company Metadata

## Garantir a existência da chave "PM_name_format" para Planos de Monitoramento

In [54]:
company.metadata["PM_name_format"] = '{prefixo}-PM-{anoCompleto}.{serialAno:04}'

## Garantir a existência da chave "CM_name_format" para Ciclos de Monitoramento

In [55]:
company.metadata["CM_name_format"] = '{prefixo}-CM-{anoCompleto}.{serialAno:04}'

## Garantir a existência da chave "COM_name_format" para Coletas

In [56]:
company.metadata["COM_name_format"] = '{prefixo}-COM-{anoCompleto}.{serialAno:04}'

In [57]:
company.save()

# Monitoring Status

In [3]:
from apps.service_orders.models import status_types

In [41]:
status_names = [
    ('Aguardando Início', 'start', False),
    ('Em Edição', 'edit', False),
    ('Em Operação', 'final', True)
]

In [42]:
for i, (status_name, status_metadata, status_flag) in enumerate(status_names):
    status = ServiceOrderActionStatus.objects.create(
        name=status_name,
        kind=status_types.MONITORING_STATUS,
        metadata={'status': status_metadata},
        is_final=status_flag,
    )
    ServiceOrderActionStatusSpecs.objects.create(
        status=status,
        company=company,
        order=i
    )

## Update todos os Planos de Monitoramento existentes com o status de Em Operação

In [96]:
final_status = ServiceOrderActionStatusSpecs.objects.filter(
    company=company,
    status__kind=status_types.MONITORING_STATUS,
    status__is_final=True,
).first().status

In [97]:
MonitoringPlan.objects.filter(
    company=company
).update(status=final_status)

12

# Create "Aguardando Avaliação" Status

In [13]:
status_filter = {
    "company": company,
    "status__kind": status_types.OCCURRENCE_RECORD_STATUS,
}
current_statuses = ServiceOrderActionStatusSpecs.objects.filter(
    **status_filter
)

In [8]:
new_order = len(current_statuses) + 1

In [61]:
aguardando_avaliacao = ServiceOrderActionStatus.objects.create(
    kind=status_types.OCCURRENCE_RECORD_STATUS,
    name="Aguardando Avaliação",
)

ServiceOrderActionStatusSpecs.objects.create(
    status=aguardando_avaliacao,
    company=company,
    order=new_order
)

<ServiceOrderActionStatusSpecs: [Aguardando Avaliação] - PRE-UHE-ITA>

## Update lock_occurrence_record_at in company Metadata

In [107]:
lock_occurrence_record_at = company.metadata.get("lock_occurrence_record_at", [])
lock_occurrence_record_at.append(str(aguardando_avaliacao.pk))
company.metadata["lock_occurrence_record_at"] = lock_occurrence_record_at
company.save()

# Approval Flow

In [14]:
# Get all status
aguardando_homologacao = str(ServiceOrderActionStatusSpecs.objects.get(
    **status_filter,
    status__name__icontains="aguardando homologa",
).status.pk)
registro_indeferido = str(ServiceOrderActionStatusSpecs.objects.get(
    **status_filter,
    status__name__icontains="registro indeferido",
).status.pk)
necessita_revisao = str(ServiceOrderActionStatusSpecs.objects.get(
    **status_filter,
    status__name__icontains="necessita revis",
).status.pk)
registro_em_elaboracao = str(ServiceOrderActionStatusSpecs.objects.get(
    **status_filter,
    status__name__icontains="registro em elabora",
).status.pk)
registro_deferido = str(ServiceOrderActionStatusSpecs.objects.get(
    **status_filter,
    status__name__icontains="registro deferido",
).status.pk)
aguardando_avaliacao = str(ServiceOrderActionStatusSpecs.objects.get(
    **status_filter,
    status__name__icontains="aguardando avalia",
).status.pk)

In [15]:
# Create ApprovalFlow
approval_flow = ApprovalFlow.objects.create(
    company=company,
    name="Fluxo Registros de Monitoramento",
    target_model="occurrence_records.OccurrenceRecord",
)

In [16]:
# Create ApprovalSteps
steps_names = [
    ("Registro em Elaboração", {"filter": ["executers"]}),
    ("Aguardando Avaliação", {"filter": ["evaluators"], "enable_edit": True}),
    ("Necessita Revisão", {"filter": ["executers"]}),
    ("Registro Indeferido", {}),
    ("Aguardando Homologação", {"filter": ["approvers", "homologator"], "enable_edit": True}),
    ("Registro Deferido", {}),
]

for step, field_options in steps_names:
    ApprovalStep.objects.create(
        name=step,
        approval_flow=approval_flow,
        field_options=field_options,
    )

In [17]:
# Create ApprovalTransitions

# Transition: Registro em Elaboração -> Aguardando Avaliação
transition_1 = {
    "name": "Solicitar Avaliação",
    "order": 2,
    "origin_id": ApprovalStep.objects.get(
        name="Registro em Elaboração",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Aguardando Avaliação",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'assignment',
        'color': 'primary',
        'variant': 'raised'
    },
    "condition": {"===": [{"var": "request.action"}, "Solicitar Avaliação"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": aguardando_avaliacao
                }
            }
        }]
    }
}

# Transition: Aguardando Avaliação -> Aguardando Homologação
transition_2 = {
    "name": "Avaliar",
    "order": 1,
    "origin_id": ApprovalStep.objects.get(
        name="Aguardando Avaliação",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Aguardando Homologação",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'done_all',
        'color': 'primary',
        'variant': 'raised'
    },
    "condition": {"===": [{"var": "request.action"}, "Avaliar"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": aguardando_homologacao
                }
            }
        }]
    }
}

# Transition: Aguardando Avaliação -> Necessita Revisão
transition_3 = {
    "name": "Solicitar Revisão",
    "order": 2,
    "origin_id": ApprovalStep.objects.get(
        name="Aguardando Avaliação",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Necessita Revisão",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'warning',
        'color': 'amber',
        'variant': 'outlined',
    },
    "condition": {"===": [{"var": "request.action"}, "Solicitar Revisão"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": necessita_revisao
                }
            }
        }]
    }
}

# Transition: Aguardando Avaliação -> Registro Indeferido
transition_4 = {
    "name": "Rejeitar",
    "order": 3,
    "origin_id": ApprovalStep.objects.get(
        name="Aguardando Avaliação",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Registro Indeferido",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'cancel',
        'color': 'red',
        'variant': 'outlined'
    },
    "condition": {"===": [{"var": "request.action"}, "Rejeitar"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": registro_indeferido
                }
            }
        }]
    }
}

# Transition: Necessita Revisão -> Aguardando Avaliação
transition_5 = {
    "name": "Solicitar Avaliação",
    "order": 1,
    "origin_id": ApprovalStep.objects.get(
        name="Necessita Revisão",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Aguardando Avaliação",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'assignment',
        'color': 'primary',
        'variant': 'raised'
    },
    "condition": {"===": [{"var": "request.action"}, "Solicitar Avaliação"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": aguardando_avaliacao
                }
            }
        }]
    }
}

# Transition: Aguardando Homologação -> Registro Deferido
transition_6 = {
    "name": "Homologar",
    "order": 1,
    "origin_id": ApprovalStep.objects.get(
        name="Aguardando Homologação",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Registro Deferido",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'done_all',
        'color': 'primary',
        'variant': 'raised'
    },
    "condition": {"===": [{"var": "request.action"}, "Homologar"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": registro_deferido
                }
            }
        }],
        "is_approved": True,
    }
}

# Transition: Aguardando Homologação -> Necessita Revisão
transition_7 = {
    "name": "Solicitar Revisão",
    "order": 2,
    "origin_id": ApprovalStep.objects.get(
        name="Aguardando Homologação",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Necessita Revisão",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'warning',
        'color': 'amber',
        'variant': 'outlined',
    },
    "condition": {"===": [{"var": "request.action"}, "Solicitar Revisão"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": necessita_revisao
                }
            }
        }]
    }
}

# Transition: Aguardando Homologação -> Registro Indeferido
transition_8 = {
    "name": "Rejeitar",
    "order": 3,
    "origin_id": ApprovalStep.objects.get(
        name="Aguardando Homologação",
        approval_flow=approval_flow,
    ).pk,
    "destination_id": ApprovalStep.objects.get(
        name="Registro Indeferido",
        approval_flow=approval_flow,
    ).pk,
    "button": {
        'icon': 'cancel',
        'color': 'red',
        'variant': 'outlined'
    },
    "condition": {"===": [{"var": "request.action"}, "Rejeitar"]},
    "callback": {
        "change_fields": [{
            "name": "status",
            "value": {
                "type": "object",
                "model": "service_orders.ServiceOrderActionStatus",
                "filters": {
                    "uuid": registro_indeferido
                }
            }
        }]
    }
}

In [18]:
all_transitions = [
    transition_1,
    transition_2,
    transition_3,
    transition_4,
    transition_5,
    transition_6,
    transition_7,
    transition_8
]

In [19]:
for item in all_transitions:
    ApprovalTransition.objects.create(**item)